# Introduction to GMSO's base data structures

In this notebook, we will explore the base data strucures, pythonic features and design decisions for the `gmso` package.

Particularly, we will cover the following aspects of the package:

1. The abstract base classes module
2. Extension of abc: Example implementing a new Bead
3. Core Classes:
    - Sites
    - Connections
    - Potentials
    - Topology
    - ForceField
4. Module gmso.lib, gmso.formats, gmso.external, gmso.lib
4. Potential and Units
7. Typed Water System
5. XML Schema for GMSO Forefield (focus on unyts(yt.unyt) and functional form (sympy)
6. Limitations, future plans

## Module gmso.abc
This module provides the abstract base classes for all other core data structures used in gmso. Our abstract base classes inherit from [pydantic](https://pydantic-docs.helpmanual.io/)'s `BaseModel` class which provides type hints as well as runtime data validation together with out-of-the-box serialization. The module structure is as follows:
```
gmso/abc 
├── abstract_connection.py 
├── abstract_potential.py 
├── abstract_site.py 
├── gmso_base.py 
```


1. [`gmso_base.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/gmso_base.py): Defines the class `GMSOBase` i.e. The base class for all our other classes that tweaks pydantic's `BaseModel` class to provide an ID based hasing as well as inject's numpydoc style docstrings from the fields of the class.

1. [`abstract_site.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_site.py): Defines the `Site` class which provides a plain topology site with following features: (a.) name (b.) position (c.) label

1. [`abstract_potential.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_potential.py): Defines the `AbstractPotential` class which is the base class for our `ParametricPotentials` as well as `PotentialTemplates`.

1. [`abstract_connection.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/abc/abstract_site.py): Defines the `AbstractConnection` class which is the base class for our `Bond`, `Angle`, `Dihedral` and `Improper` classes.

## Example Implementation: Bead

The `Bead` class can now be implemented as a subclass of the abstract `Site` class. We can use the existing attributes from the super class like `name`, `position` etc... and define new attributes and methods for `Bead`. The goal is the consolidation of as many universal characteristics of a generic topology site into a base class (`Site`) and tweak its down-stream usage according to the needs of a particular site(like an `Atom` or a `Bead`). Usage of `Site` to create an `Bead` class is shown below:

In [23]:
import warnings
warnings.simplefilter('ignore')
import unyt as u
from pydantic import Field

from gmso.abc.abstract_site import Site


class Bead(Site):
    __base_doc__ = "Basic Bead class inheriting from the Site Class"
    mass_: u.unyt_quantity = Field(
        default=1.0*u.amu
    )
    
    class Config:
        fields = {
            'mass_': 'mass'
        }
        alias_to_fields = {
            'mass': 'mass_'
        }
    
my_bead = Bead()
my_bead.name  # When you inherit, the attribute(field) `name` is injected as the class name(Bead in this case)

'Bead'

In [24]:
# Documentation is injected automatically as well
%pdoc Bead

## Core Classes
In `gmso` we define the following core classes. All of our core classes make use of `gmso.abc` to define a particular site, connection or potential. The module `gmso.core`'s structure is as follows:

```
gmso/core/
├── angle.py
├── angle_type.py
├── atom.py
├── atom_type.py
├── bond.py
├── bond_type.py
├── box.py
├── dihedral.py
├── dihedral_type.py
├── element.py
├── forcefield.py
├── improper.py
├── improper_type.py
├── parametric_potential.py
└── topology.py
```


### Sites
1. [`atom.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/atom.py): Defines the class `gmso.core.atom.Atom` which inherits from `gmso.abc.abstract_site.Site` to define an `Atom`.

In [3]:
import json
import unyt as u
class UnytJsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, u.unyt_array):
            return {
                    'array': obj.ravel().tolist(),
                    'unit': str(obj.units)    
            }
        
        return json.JSONEncoder.default(self, obj)

In [4]:
from gmso.core.atom import Atom
from pprint import pprint
atom1 = Atom(name='atom1', charge=2.0*u.elementary_charge)
atom2 = Atom(name='atom2', charge=1.0*u.elementary_charge)

# Dumping the model as json is easy
pprint(json.dumps(atom1.dict(by_alias=True, exclude_unset=True), cls=UnytJsonEncoder, indent=2))

('{\n'
 '  "name": "atom1",\n'
 '  "charge": {\n'
 '    "array": [\n'
 '      3.2043532416e-19\n'
 '    ],\n'
 '    "unit": "C"\n'
 '  }\n'
 '}')



### Connections
1. [`angle.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/angle.py): Defines the class `gmso.core.angle.Angle` which inherits from `gmso.abc.abstract_connection.Connection` to define a 3-partner connection between `Atoms`.

2. [`bond.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/bond.py): Defines the class `gmso.core.bond.Bond` which inherits from `gmso.abc.abstract_connection.Connection` to define a 2-partner connection between `Atoms`.

3. [`dihedral.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/dihedral.py): Defines the class `gmso.core.dihedral.Dihedral` which inherits from `gmso.abc.abstract_connection.Connection` to define a 4-partner connection(dihedral) between `Atoms`.

4. [`improper.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/improper.py): Defines the class `gmso.core.improper.Improper` which inherits from `gmso.abc.abstract_connection.Connection` to define a 4-partner connection(improper) between `Atoms`.

In [5]:
from gmso import Bond
bond = Bond(connection_members=[atom1, atom2])
bond.connection_members

(<Atom, id 140099994746304>, <Atom, id 140100039301152>)


### Potentials
1. [`parametric_potential.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/parametric_potential.py): Defines the class `gmso.core.parametric_potential.ParametricPotential` which inherits from `gmso.abc.abstract_potential.Potential` to define a `ParametricPotential` class which stores the functional form of a Potential as sympy expression and parameters of the potential expression as `unyt_quantities`.

2. [`atom_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/atom_type.py): Defines the class `gmso.core.atom_type.AtomType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for an `AtomType`.

3. [`bond_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/bond_type.py): Defines the class `gmso.core.bond_type.BondType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for a `BondType`.

4. [`angle_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/angle_type.py): Defines the class `gmso.core.angle_type.AngleType` which inherits from `gmso.core.parametric_potential.ParametricPotential` to describe properties for an `AngleType`.

5. [`dihedral_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/dihedral_type.py) and [`improper_type.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/atom_type.py): Defines the classes `gmso.core.atom_type.DihedralType` and `gmso.core.improper_type.ImproperType` which inherit from `gmso.core.parametric_potential.ParametricPotential` which describe properties for a `DihedralType` and `ImproperType` respectively.

In [6]:
from gmso.core.parametric_potential import ParametricPotential

# Handle potential expression using separate Expression class
new_potential = ParametricPotential(
            name='mypotential',
            expression='a*x+b',
            parameters={
                'a': 1.0*u.g,
                'b': 1.0*u.m
            },
            independent_variables={'x'}
)

try:
    new_potential.independent_variables = 'y'
except ValueError as e:
    print(e)

symbol y is not in expression's free symbols. Cannot use an independent variable which doesn't exist in the expression's free symbols {x, a, b}


### Topologies
1. [`topology.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/topology.py): Defines the class `gmso.core.topology.Topology` which is the main data structure responsible for interactions between various potentials, connections and sites to form a chemical topology representation.

In [7]:
from gmso import Topology, Atom, Bond
methane_top = Topology()
c = Atom(name='c')
h1 = Atom(name='h1')
h2 = Atom(name='h2')
h3 = Atom(name='h3')
h4 = Atom(name='h4')
ch1 = Bond(connection_members=[c,h1])
ch2 = Bond(connection_members=[c,h2])
ch3 = Bond(connection_members=[c,h3])
ch4 = Bond(connection_members=[c,h4])
methane_top.add_site(c, update_types=False)
methane_top.add_site(h1, update_types=False)
methane_top.add_site(h2, update_types=False)
methane_top.add_site(h3, update_types=False)
methane_top.add_site(h4, update_types=False)
methane_top.add_connection(ch1, update_types=False
)
methane_top.add_connection(ch2, update_types=False)
methane_top.add_connection(ch3, update_types=False)
methane_top.add_connection(ch4, update_types=False)
methane_top.update_topology()
print(methane_top)

<Topology 5 sites, 4 connections, id: 140099994446928>


### Forcefield
1. [`forcefield.py`](https://github.com/mosdef-hub/gmso/blob/3ff3829cb4bc492b41e5e520d26d35c09c5338a4/gmso/core/forcefield.py): Defines the class `ForceField` which defines the in memory representation of the gmso forcefield schema. An Example schema `tip3p.xml` is shown below:

```xml
<?xml version='1.0' encoding='UTF-8'?>
<ForceField name="TIP3P" version="0.0.1"> 
  <!-- Defines metadata as units -->
  <FFMetaData>
    <Units energy="kJ/mol" mass="amu" charge="elementary_charge" distance="nm"/>
  </FFMetaData>
  <!-- Potentials can be grouped together by expression and can have optional names -->
  <AtomTypes expression="4*epsilon * ((sigma/r)**12 - (sigma/r)**6)">
     <!--   Units for parameters are defined in this tag    -->
    <ParametersUnitDef parameter="epsilon" unit="kJ/mol"/>
    <ParametersUnitDef parameter="sigma" unit="nm"/>
    <AtomType name="opls_111" element="O" charge="-0.834" mass="16" definition="O" description="water O">
      <Parameters>
        <Parameter name="epsilon" value="0.636386"/>
        <Parameter name="sigma" value="0.315061"/>
      </Parameters>
    </AtomType>
    <AtomType name="opls_112" element="H" charge="0.417" mass="1.011" definition="H">
      <Parameters>
        <Parameter name="epsilon" value="0.0"/>
        <Parameter name="sigma" value="1.0"/>
      </Parameters>
    </AtomType>
  </AtomTypes>
  <BondTypes expression="0.5 * k * (r-r_eq)**2">
    <ParametersUnitDef parameter="k" unit="kJ/mol/nm**2"/>
    <ParametersUnitDef parameter="r_eq" unit="nm"/>
    <BondType name="BondType-Harmonic-1" type1="opls_111" type2="opls_112">
      <Parameters>
        <Parameter name="k" value="502416.0"/>
        <Parameter name="r_eq" value="0.09572"/>
      </Parameters>
    </BondType>
  </BondTypes>
  <AngleTypes expression="0.5 * k * (theta - theta_eq)**2">
    <ParametersUnitDef parameter="k" unit="kJ/(mol*radian**2)"/>
    <ParametersUnitDef parameter="theta_eq" unit="radian"/>
    <AngleType name="AngleType-Harmonic-1" type1="opls_112" type2="opls_111" type3="opls_112">
      <Parameters>
        <Parameter name="k" value="682.02"/>
        <Parameter name="theta_eq" value="1.824218134"/>
      </Parameters>
    </AngleType>
  </AngleTypes>
</ForceField>
```

In [8]:
from gmso import ForceField 
from gmso.tests.utils import get_path
water = ForceField(get_path('tip3p.xml'))
water.bond_types

{'opls_111~opls_112': <BondType BondType-Harmonic-1, id 140099994743984>}

## Module gmso.lib, gmso.formats, gmso.external
Module gmso.lib defines the a lazy loading module from json files. For example, the `OPLSTorsionPotential` is defined as a json file as shown below. The `PotentialTemplate` class also inherits from `gmso.abc.abstract_potential.Potential` is wrapped by a singleton class and is immutable.

```json
{
  "name": "OPLSTorsionPotential",
  "expression": "0.5 * k0 + 0.5 * k1 * (1 + cos(phi)) + 0.5 * k2 * (1 - cos(2*phi)) + 0.5 * k3 * (1 + cos(3*phi)) + 0.5 * k4 * (1 - cos(4*phi))",
  "independent_variables": "phi"
}

```

In [9]:
from gmso.lib.potential_templates import PotentialTemplateLibrary
opls = PotentialTemplateLibrary()['OPLSTorsionPotential']
opls.dict(by_alias=True)

{'name': 'OPLSTorsionPotential',
 'potential_expression': <PotentialExpression, expression: 0.5*k0 + 0.5*k1*(cos(phi) + 1) + 0.5*k2*(1 - cos(2*phi)) + 0.5*k3*(cos(3*phi) + 1) + 0.5*k4*(1 - cos(4*phi)), 1 independent variables>}

Modules `gmso.formats` and `gmso.external` define file writers to different simulation engines and converters to/from external packages.

## Handling Potential Parameters and Units in GMSO

Potential expression and parameters represent the functional form of any bonded or non bonded potential in GMSO. At its core, any potential class (inheriting from `gmso.abstract_potential.AbstractPotential`) is a container for three entities:

1. The expression for the Potential
2. The parameters (i.e. non-free variables) in the Potential expression and their values
3. The Potential reference (for example atom_type or atom_class for an AtomType)

Basically, we delegate the handling of potential expression and units in GMSO to a utility class called `PotentialExpression`, which keeps track between changes of independent variables and expression of a potential expression. Basically, we use the same utility class to manage both parametric and non-parametric potential Expression.

In [10]:
from gmso.utils.expression import _PotentialExpression

expression = _PotentialExpression(
    expression='x*b+c',
    independent_variables={'c'}
)

expression.is_parametric

try:
    expression.independent_variables = 'd' # Will throw an error
except ValueError as e:
    print(e)

symbol d is not in expression's free symbols. Cannot use an independent variable which doesn't exist in the expression's free symbols {x, c, b}


As shown above, we can use the `PotentialTemplateLibrary` to parametrize any potential we would want. All the parameters are maintained as `unyt_arrays` which makes them easier to work with in any unit system. An example of creating a Potential from LJ template is shown below:

In [11]:
import unyt as u
from gmso.core.parametric_potential import ParametricPotential
from gmso.lib.potential_templates import PotentialTemplateLibrary
pt_lib = PotentialTemplateLibrary()
pt_lib.get_available_template_names()

('HarmonicTorsionPotential',
 'MiePotential',
 'HarmonicImproperPotential',
 'LennardJonesPotential',
 'HarmonicAnglePotential',
 'RyckaertBellemansTorsionPotential',
 'BuckinghamPotential',
 'PeriodicTorsionPotential',
 'OPLSTorsionPotential',
 'HarmonicBondPotential')

In [12]:
pt_lib['LennardJonesPotential'].expression

4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)

In [13]:
pt_lib['LennardJonesPotential'].independent_variables

{r}

In [14]:
lj_parametrized = ParametricPotential.from_template(
    potential_template=pt_lib['LennardJonesPotential'],
    parameters={
        'sigma' : 1.0 * u.nm,
        'epsilon': 1.0 * u.elementary_charge
    }
)

lj_parametrized.parameters

{'sigma': unyt_quantity(1., 'nm'),
 'epsilon': unyt_quantity(1.60217662e-19, 'C')}

In [15]:
lj_parametrized.expression

4*epsilon*(-sigma**6/r**6 + sigma**12/r**12)

## Example: Typed Water System using mBuild

In [16]:
import mbuild as mb
from gmso.core.angle import Angle
from gmso.core.forcefield import ForceField
from gmso.tests.utils import get_path
from gmso.external import from_mbuild

water = mb.load(get_path('tip3p.mol2'))
water.name = 'water'
water[0].name = 'opls_111'
water[1].name = water[2].name = 'opls_112'

packed_water = mb.fill_box(
    compound=water,
    n_compounds=2,
    box=mb.Box([2, 2, 2])
)

water_top = from_mbuild(packed_water)

ff = ForceField(get_path('tip3p.xml'))

element_map = { "O": "opls_111", "H": 'opls_112'}


for atom in water_top.sites:
    atom.atom_type = ff.atom_types[atom.name]

for bond in  water_top.bonds:
    bond.bond_type = ff.bond_types[f'opls_111~opls_112']
    
for subtop in water_top.subtops:
    angle = Angle(
        connection_members=[site for site in subtop.sites],
        name='opls_112~opls_111~opls_112',
        angle_type=ff.angle_types["opls_112~opls_111~opls_112"]
    )
    water_top.add_connection(angle)

water_top.update_topology()

## XML Schema for gmso Forcefield

We use XML standard 1.0 and enforce strict validation (no extra attributes or new patterns allowed) via `lxml` and custom logic to create a forcefield schema with the following major tags:

1. Metadata tag `<FFMetadata/>`: This tag can be used to default units for the forcefield schema. We can also save the `electrostaticScale` as well as `nonBonded14Scale` for the forcefield in the metadata tag.
2. Potential Group Tags `<AtomTypes/>`, `<BondTypes/>`, `<AngleTypes/>`, `<DihedralTypes/>`. These tags can be used to group togehter potentials with similar attributes like expression and parameters. They consolidate information regarding individual poetential. A forcefeild schema can have multiple group tags of the same kind.
3. Individual Potentials `<AtomType/>`, `<BondType/>`, `<AngleType/>`, `<DihedralType/>` and `<ImproperType/>` contains attributes and subtags to represent individual potentials.

The snippet below shows the rules for representing an `<AtomType>` element.

```xsd
 <xsd:complexType name="AtomType">
        <xsd:sequence>
            <xsd:element ref="Parameters" minOccurs="0" maxOccurs="1"/>
        </xsd:sequence>
        <xsd:attribute name="name" type="xsd:string" use="optional"/>
        <xsd:attribute name="mass" type="xsd:float" use="optional"/>
        <xsd:attribute name="element" type="xsd:string" use="optional"/>
        <xsd:attribute name="charge" type="xsd:float" use="optional"/>
        <xsd:attribute name="expression" type="xsd:string" use="optional"/>
        <xsd:attribute name="independent_variables" type="xsd:string" use="optional"/>
        <xsd:attribute name="atomclass" type="xsd:string" use="optional"/>
        <xsd:attribute name="doi" type="xsd:string" use="optional"/>
        <xsd:attribute name="overrides" type="xsd:string" use="optional"/>
        <xsd:attribute name="definition" type="xsd:string" use="optional"/>
        <xsd:attribute name="description" type="xsd:string" use="optional"/>
    </xsd:complexType>
```

As shown above, when creating a `Forcefield` object from the serialized representation of a `forcefield`, the strings for expression are converted into symbols using the `sympy` package and units are handled by yt's `unyt` package.

## Limitations and Future Plans